In [197]:
import time
import csv
import re
import requests
import json
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument("disable-gpu")
options.add_argument('--kiosk')

driver_path = 'D:/JISUNG/jupyter/driver/chromedriver.exe'
browser = webdriver.Chrome(driver_path, chrome_options=options)
browser.get('http://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do')
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="content"]/div[3]/div[2]/a[1]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="sPrdtStatStr"]'))
time.sleep(1)
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="mul_chk_det0"]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="layerConfirmChk"]'))
time.sleep(1)
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="sGenreStr"]'))
time.sleep(1)
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="chkAllChkBox"]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="mul_chk_det18"]'))
browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="layerConfirmChk"]'))

browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="searchForm"]/div[1]/div[5]/button[1]'))
time.sleep(1)
max_count = int(browser.find_element_by_xpath('//*[@id="content"]/div[4]/div[1]/span/em').text.replace(',', ''))

file = open('movie.csv', 'wt', encoding='utf-8', newline='')
writer = csv.writer(file)
writer.writerow(
    [
        'title', # 제목
        'audience', # 관객수(종속변수)
        'release_date', # 개봉일
        'country', # 제작 국가
        'director', # 감독
        'producer', # 제작사
        'actor1', # 주연1
        'actor2', # 주연2
        'actor3', # 주연3
        'genre', # 장르
        'screen', # 스크린 수
        'running_time', # 상영시간
        'Rating', # 등급
        'holiday', # 개봉일부터 한 달 동안의 휴일 수
    ]
)

page = 1
for i in range(1, max_count + 1):
    title = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a').text.strip()
    country = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[5]/span').text.strip()
    director = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[9]/span/a').text.strip()
    producer = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[10]/span/a').text.strip()
    temp = browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[7]/span').text.strip()
    genre = re.compile(r'[가-힣]+').search(temp).group().strip()
    
    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="content"]/div[4]/table/tbody/tr[{i % 10 if i % 10 else 10}]/td[1]/span/a'))
    time.sleep(5)
    temp = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[4]').text.split('|')
    running_time = temp[3].strip().split('분')[0]
    Rating = temp[4].strip()
    if browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dt[5]').text.strip() == '개봉일':
        release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[5]').text.strip()
    else:
        release_date = browser.find_element_by_xpath(f'//*[@class="ovf cont"]/dd[6]').text.strip()

    for j in range(1, 4):
        try:
            if browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dt').text.strip() == '배우':
                try:
                    temp = f'browser.find_element_by_xpath(\'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]\').text.strip()'
                    t = browser.find_element_by_xpath(f'//*[@class="staffMore"]/dl/div[2]/dd/table[1]/tbody/tr/td/a[{j}]').text.strip()
                    if re.compile(r"\(").search(t) == None:
                        exec(f'actor{j} = {temp}')
                    else:
                        exec(f'actor{j} = re.compile(r"[가-힣]+").search({temp}).group().strip()')
                except:
                    exec(f'actor{j} = actor{j+1} = actor{j+2} = ""')
                    break
            else:
                actor1 = actor2 = actor3 = ''
                break
        except:
            actor1 = actor2 = actor3 = ''
            break

    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="wrap_tab tab2"]/ul/li[2]'))
    time.sleep(5)
    temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[2]').text
    screen = re.compile(r'\d+\n').search(temp).group().strip()
    temp = browser.find_element_by_xpath(f'//*[@class="info info2"]/*[@class="tbl_comm"]/tfoot/tr/td[4]').text
    audience = re.compile(r'[\d]+\s').search(temp.replace(',', '')).group().strip()
    
    browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@class="hd_layer"]/a[2]'))
    
    temp = release_date.split('-')
    year = temp[0]
    month = temp[1]
    day = temp[2]
    date = datetime.date(int(year), int(month), int(day))

    holiday = 0
    for _ in range(31):
        d = (date).weekday()

        if date.year == int(year) and date.month == int(month):
            url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={year}{month}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'
        else:
            url = f'https://m.search.naver.com/p/csearch/content/qapirender.nhn?where=nexearch&key=CalendarAnniversary&pkid=134&q={date.year}{"0" + str(date.month) if len(str(date.month)) == 1 else str(date.month)}%EC%9B%94&_callback=window.__jindo2_callback._directsearch_calendar_famous_month_0'

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        data = re.compile(r'[(](.*?)[)]').search(str(soup)).group().replace('(', '').replace(')', '')
        parse = json.loads(data)

        day_off = 0
        for k in parse['openCalendar']['daysList']:
            if k['solarDate'] == str(date).replace('-', ''):
                if k['dayOff'] == 'true':
                    day_off = 1
                    break

        if d == 5 or d == 6 or day_off:
            holiday += 1
        date += datetime.timedelta(days=1)
    
    if not i % 10:
        if page % 10:
            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath(f'//*[@id="pagingForm"]/div/ul/li[{10 if page % 10 == 9 else (page + 1) % 10}]/a'))
        else:
            browser.execute_script("arguments[0].click();", browser.find_element_by_xpath('//*[@id="pagingForm"]/div/a[3]'))
        page += 1
        time.sleep(2)

    writer.writerow(
        [
            title,
            audience,
            release_date,
            country,
            director,
            producer,
            actor1,
            actor2,
            actor3,
            genre,
            screen,
            running_time,
            Rating,
            holiday
        ]
    )
    
file.close()
browser.quit()

C:\Users\jejsm\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


KeyboardInterrupt: 